In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from osgeo import gdal, osr
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import scipy.io as sio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as matplotlib

In [ ]:
CA_keras_stockdon_comp = sio.loadmat('CA_keras_stockdon_comp_historic.mat')
slope_CA_all = np.squeeze(CA_keras_stockdon_comp['slope_CA_all'])
std_shore_r2 = np.squeeze(CA_keras_stockdon_comp['std_shore_r2'])
std_shore = np.squeeze(CA_keras_stockdon_comp['std_shore'])
std_tide = np.squeeze(CA_keras_stockdon_comp['std_tide'])

In [ ]:
df = pd.DataFrame({
    'r2_dnn': std_shore_r2[:,0]/std_shore,
    'r2_stock': std_shore_r2[:,1]/std_shore,
    'r2_rugg': std_shore_r2[:,2]/std_shore,
    'r2_beuz': std_shore_r2[:,3]/std_shore,
    'r2_holm': std_shore_r2[:,4]/std_shore,
    'r2_powe': std_shore_r2[:,5]/std_shore,
    'r2_ipa': std_shore_r2[:,6]/std_shore,
    'tide_corr': std_tide/std_shore,
    'slope_CA_all': slope_CA_all
})

font_size = 12
matplotlib.rcParams['font.size'] = font_size

fig, (ax1,ax0) = plt.subplots(nrows=1,ncols=2)
plt.subplots_adjust(wspace=0.05)
plt.subplots_adjust(hspace=1)
fig.set_size_inches(12, 5)

# Define bins for slope_CA_all in 0.02 increments up to 0.2
bins = np.arange(0.02, 0.17, 0.02)

# Bin the slope_CA_all data
df['slope_bins'] = pd.cut(df['slope_CA_all'], bins=bins, include_lowest=True)

sns.set(style="whitegrid")  # Set style for the plot if you wish

# Group by the bins and calculate mean and standard deviation
grouped_dnn = df.groupby('slope_bins')['r2_dnn'].agg(['mean', 'std', 'count']).reset_index()
grouped_dnn['slope_bins'] = grouped_dnn['slope_bins'].astype(str)
ax0.plot(grouped_dnn['slope_bins'], grouped_dnn['mean'], marker='o', color=sns.color_palette("Paired")[3], markersize=5,label='DNN (This Study)')

grouped_dnn = df.groupby('slope_bins')['r2_rugg'].agg(['mean', 'std', 'count']).reset_index()
grouped_dnn['slope_bins'] = grouped_dnn['slope_bins'].astype(str)
ax0.plot(grouped_dnn['slope_bins'], grouped_dnn['mean'], marker='o',color=sns.color_palette("Paired")[6], markersize=5,label='Ruggiero et al., (2001)')

grouped_dnn = df.groupby('slope_bins')['r2_powe'].agg(['mean', 'std', 'count']).reset_index()
grouped_dnn['slope_bins'] = grouped_dnn['slope_bins'].astype(str)
ax0.plot(grouped_dnn['slope_bins'], grouped_dnn['mean'], marker='o',color=sns.color_palette("Paired")[9], markersize=5, label='Power et al., (2018)')

grouped_dnn = df.groupby('slope_bins')['r2_stock'].agg(['mean', 'std', 'count']).reset_index()
grouped_dnn['slope_bins'] = grouped_dnn['slope_bins'].astype(str)
ax0.plot(grouped_dnn['slope_bins'], grouped_dnn['mean'], marker='o',color=sns.color_palette("Paired")[1], markersize=5,label='Stockdon et al., (2006)')

grouped_dnn = df.groupby('slope_bins')['r2_holm'].agg(['mean', 'std', 'count']).reset_index()
grouped_dnn['slope_bins'] = grouped_dnn['slope_bins'].astype(str)
ax0.plot(grouped_dnn['slope_bins'], grouped_dnn['mean'], marker='o',color=sns.color_palette("Paired")[7], markersize=5, label='Holman, (1986)')

# grouped_dnn = df.groupby('slope_bins')['r2_ipa'].agg(['mean', 'std', 'count']).reset_index()
# grouped_dnn['slope_bins'] = grouped_dnn['slope_bins'].astype(str)
# ax0.plot(grouped_dnn['slope_bins'], grouped_dnn['mean'], marker='o',color=sns.color_palette("Paired")[7], markersize=5, label='IPA')

grouped_dnn = df.groupby('slope_bins')['tide_corr'].agg(['mean', 'std', 'count']).reset_index()
grouped_dnn['slope_bins'] = grouped_dnn['slope_bins'].astype(str)
ax0.plot(grouped_dnn['slope_bins'], grouped_dnn['mean'], marker='o',color=[0.4,0.4,0.4], markersize=5, linestyle='--', label='Tide Only')
ax0.grid(True)

for i, row in grouped_dnn.iterrows():
    ax0.text(row['slope_bins'], 1.01, 'N='+str(row['count']), color='black', ha="center")

ax0.legend()
ax0.tick_params(axis='x', labelrotation = 45)

# Optional: Set the labels and title
ax0.set_xlabel('Slope Bins')
ax0.set_ylabel(r'$\frac{RMSE(X_{s,corr})}{STD({X_{s}})}$',fontsize=22)
ax0.set_ylim([0.6,1])
ax0.text(5.9,0.61,'(b)',fontsize=14)

# Improve the x-axis labels readability
CA_keras_beach_width = sio.loadmat('CA_keras_beach_width.mat')
mse_field = np.squeeze(CA_keras_beach_width['mse_field'])
slope_CA_all_field = np.squeeze(CA_keras_beach_width['slope_CA_all'])
width_std = np.squeeze(CA_keras_beach_width['width_std'])
shore_std = np.squeeze(CA_keras_beach_width['shore_std'])
mse_field_r2tide = np.squeeze(CA_keras_beach_width['mse_field_r2tide'])
mse_field_r2stock = np.squeeze(CA_keras_beach_width['mse_field_r2stock'])
mse_field_r2stock_only = np.squeeze(CA_keras_beach_width['mse_field_r2stock_only'])
mse_field_none = np.squeeze(CA_keras_beach_width['mse_field_none'])

df = pd.DataFrame({
    'mse_field': mse_field/mse_field_none,
    'mse_field_r2stock': mse_field_r2stock/mse_field_none,
    'mse_field_r2stock_only': mse_field_r2stock_only/mse_field_none,
    'mse_field_r2tide': mse_field_r2tide/mse_field_none,
    'mse_field_r2none': mse_field_none/mse_field_none,
    'slope_CA_all': slope_CA_all
})

# Define bins for slope_CA_all in 0.02 increments up to 0.2
bins = np.arange(0.02, 0.17, 0.02)

# Bin the slope_CA_all data
df['slope_bins'] = pd.cut(df['slope_CA_all'], bins=bins, include_lowest=True)

# Group by the bins and calculate mean and standard deviation
# grouped_dnn = df.groupby('slope_bins')['mse_field_r2none'].agg(['median', 'std', 'count']).reset_index()
# grouped_dnn['slope_bins'] = grouped_dnn['slope_bins'].astype(str)
# ax1.plot(grouped_dnn['slope_bins'], grouped_dnn['median'], marker='o',color=[0.5,0.5,0.5], markersize=5,label='No Correction',linestyle='dashdot')

grouped_dnn = df.groupby('slope_bins')['mse_field_r2stock_only'].agg(['mean', 'std', 'count']).reset_index()
grouped_dnn['slope_bins'] = grouped_dnn['slope_bins'].astype(str)
ax1.plot(grouped_dnn['slope_bins'], grouped_dnn['mean'], marker='o', color=sns.color_palette("Paired")[7], markersize=5,label='Stockdon et al., (2006)')

grouped_dnn = df.groupby('slope_bins')['mse_field_r2tide'].agg(['mean', 'std', 'count']).reset_index()
grouped_dnn['slope_bins'] = grouped_dnn['slope_bins'].astype(str)
ax1.plot(grouped_dnn['slope_bins'], grouped_dnn['mean'], marker='o',color=[0.4,0.4,0.4], markersize=5,label='Tide Only',linestyle='--')

grouped_dnn = df.groupby('slope_bins')['mse_field_r2stock'].agg(['mean', 'std', 'count']).reset_index()
grouped_dnn['slope_bins'] = grouped_dnn['slope_bins'].astype(str)
ax1.plot(grouped_dnn['slope_bins'], grouped_dnn['mean'], marker='o',color=sns.color_palette("Paired")[1], markersize=5,label='Stockdon et al., (2006) + Tide')

grouped_dnn = df.groupby('slope_bins')['mse_field'].agg(['mean', 'std', 'count']).reset_index()
grouped_dnn['slope_bins'] = grouped_dnn['slope_bins'].astype(str)
ax1.plot(grouped_dnn['slope_bins'], grouped_dnn['mean'], marker='o', color=sns.color_palette("Paired")[3], markersize=5,label='DNN (This Study)')

ax1.set_xlabel('Slope Bins')
ax1.set_ylabel(r'$\frac{RMSE(X_{survey} - X_{s,corr})}{RMSE({X_{survey} - X_s})}$',fontsize=22)
ax1.set_ylim([0.5,1.2])

# Improve the x-axis labels readability
ax1.tick_params(axis='x', labelrotation = 45)
ax1.text(5.9,0.53,'(a)',fontsize=14)
ax1.grid(True)
for i, row in grouped_dnn.iterrows():
    ax1.text(row['slope_bins'], 1.21, 'N='+str(row['count']), color='black', ha="center")


ax1.legend()

# Show the plot
plt.tight_layout()  # Adjust layout to fit labels
plt.savefig('figures/Stockdon_Comp.png',bbox_inches='tight')
plt.show()